# Notebook : Ingestion quotidienne de données éoliennes

## 📋 Objectif
Charger de manière incrémentale les données de production éolienne depuis GitHub vers le Lakehouse Bronze.

## 🔄 Logique
1. Identifier la date la plus récente dans Bronze
2. Calculer le jour suivant (date + 1)
3. Télécharger le fichier CSV correspondant depuis GitHub
4. Ajouter les nouvelles données en mode append

## ⚙️ Exécution
- **Fréquence recommandée** : Quotidienne (via pipeline)
- **Durée moyenne** : 50-60 secondes
- **Dépendances** : Repository GitHub public accessible

## 📦 Dépendances
- Lakehouse : LH_Wind_Power_Bronze
- Source : https://github.com/gsoulat/data-training-fabric/tree/main/eolienne

In [12]:
import requests
import pandas as pd
from datetime import datetime, timedelta

StatementMeta(, 5f306106-5e0a-4325-b042-1ec2dfd48c0a, 14, Finished, Available, Finished)

In [13]:
# URL de base du repository GitHub
base_url = "https://raw.githubusercontent.com/gsoulat/data-training-fabric/main/eolienne/"

# Chemin vers la table Bronze
bronze_table_path = "abfss://WindPowerAnalytics@onelake.dfs.fabric.microsoft.com/LH_Bronze.Lakehouse/Tables/dbo/wind_power"

StatementMeta(, 5f306106-5e0a-4325-b042-1ec2dfd48c0a, 15, Finished, Available, Finished)

In [14]:
df_spark = spark.read.format("delta").load(bronze_table_path)

df_pandas = df_spark.toPandas()

StatementMeta(, 5f306106-5e0a-4325-b042-1ec2dfd48c0a, 16, Finished, Available, Finished)

In [15]:
most_recent_date = pd.to_datetime(df_pandas['date'], format ="%Y%m%d").max()

next_date = (most_recent_date + timedelta(days = 1 )).strftime("%Y%m%d")

StatementMeta(, 5f306106-5e0a-4325-b042-1ec2dfd48c0a, 17, Finished, Available, Finished)

In [16]:
print(most_recent_date)
print(next_date)

StatementMeta(, 5f306106-5e0a-4325-b042-1ec2dfd48c0a, 18, Finished, Available, Finished)

2024-06-16 00:00:00
20240617


In [17]:
file_url = f"{base_url}{next_date}_wind_power_data.csv"
print(file_url)

df_pandas_new = pd.read_csv(file_url)
df_pandas_new['date'] = pd.to_datetime(df_pandas_new['date'])

StatementMeta(, 5f306106-5e0a-4325-b042-1ec2dfd48c0a, 19, Finished, Available, Finished)

https://raw.githubusercontent.com/gsoulat/data-training-fabric/main/eolienne/20240617_wind_power_data.csv


In [18]:
df_spark_new = spark.createDataFrame(df_pandas_new, schema = df_spark.schema)
df_spark_new.write.format('delta').mode("append").save(bronze_table_path)

StatementMeta(, 5f306106-5e0a-4325-b042-1ec2dfd48c0a, 20, Finished, Available, Finished)